![CSTR Diagram](../assets/Image_README.png)
# CSTR Simulator

A modular, GUI-integrated simulator for continuous stirred-tank reactors (CSTRs) with recycle loop, temperature optimization, and real industrial case studies.

This notebook was created as a collaborative project for the **EPFL** course _Practical Programming in Chemistry (**CHE-200**)_.

---

## Introduction

youpi

---

## Why Continuous Flow Stirred Reactors?

...

## 🔧 Key Functions of the `CSTRSimulator` Class

This section presents and tests the most important built-in methods of the `CSTRSimulator` class.  
Each function plays a central role in simulating the behavior of a continuous stirred-tank reactor (CSTR), including performance evaluation, kinetics, and efficiency metrics.

For each function, you will find:
- a short explanation of why it matters,
- the Python code to test it with concrete input values,
- and the expected output.

In [23]:
import sys, os
sys.path.append(os.path.abspath("../src"))

from projet_chem200.cstr_simulator.functions import CSTRSimulator

# Création de l'instance du simulateur
sim = CSTRSimulator()

# Set the components used in your reactions (required for internal methods)
sim.components = ["A", "B"]  # You can add more if needed, like "C", "D", etc.

### 1. `calculate_conversion`

🔍 Why it's important:

Conversion tells us **how much of the initial reactant was consumed** inside the reactor.  
It is a key performance metric to assess **reactor efficiency** and **compare operating conditions**.

In [24]:
inlet = {"A": 1.0}    # Inlet concentration (mol/L)
outlet = {"A": 0.3}   # Outlet concentration (mol/L)

X = sim.calculate_conversion(inlet_conc=inlet, outlet_conc=outlet)
print("Conversion of A =", X["A"])

Conversion of A = 0.7


### 2. `calculate_yield`

🔍 Why it's important:

The yield tells us **how much of the desired product is obtained** compared to the amount of reactant consumed.  
It is critical for evaluating **reaction efficiency** and **selectivity** in industrial processes.

In [32]:
from projet_chem200.cstr_simulator.functions import CSTRSimulator

# 1. Create the simulator instance
sim = CSTRSimulator()

# 2. Define components and target product
sim.components = ["A", "B"]
sim.target_product = "B"
sim.recycle_ratio = 0.0  # Required to avoid AttributeError

# 3. Define a simple reaction: A → B
sim.reactions = [
    {
        "stoichiometry": {"A": -1, "B": 1},
        "reactants": ["A"],
        "products": ["B"]
    }
]

# 4. Set inlet and outlet concentrations
inlet = {"A": 1.0, "B": 0.0}
outlet = {"A": 0.3, "B": 0.7}

sim.feed_composition = inlet      # Required for theoretical yield calculation
sim.concentrations = outlet       # Simulated outlet concentrations

# 5. Calculate yield
Y = sim.calculate_yield(product="B")
print("Yield of B =", Y)

Yield of B = 0.7


### 3. `reaction_rate` (via `sim.rate_law`)

🔍 Why it's important:

The reaction rate determines **how fast the chemical reaction proceeds** inside the reactor.  
It is essential for **dimensioning the reactor**, calculating **product formation rates**, and analyzing **temperature or concentration effects**.  
In this simulator, the rate law is manually defined using a lambda function.

In [37]:
from projet_chem200.cstr_simulator.functions import CSTRSimulator

# 1. Create the simulator
sim = CSTRSimulator()

# 2. Define the reaction data
reaction = {
    "frequency_factor": 1e7,                # A (1/s)
    "activation_energy": 80000,            # Ea (J/mol)
    "reaction_order": {"A": 1, "B": 1},     # r = k * [A]^1 * [B]^1
}

# 3. Define concentrations and temperature
component_conc = {"A": 0.5, "B": 0.4}  # mol/L
temp = 350  # K

# 4. Call the method
r = sim.reaction_rate(component_conc, temp, reaction)
print("Reaction rate =", r, "mol/L/s")

Reaction rate = 2.297483991329831e-06 mol/L/s


### 4. `solve_steady_state`

🔍 Why it's important:

This function calculates the **steady-state concentrations** in the reactor for a given feed, temperature, volume, and recycle ratio.

It is the **core engine** of the CSTR simulator, iteratively computing the final concentrations using:
- the specified **reaction kinetics** (`reaction_rate`)
- **flow and volume conditions**
- optional **recycle stream**

It stops when the system reaches convergence or the maximum number of iterations.

In [51]:
from projet_chem200.cstr_simulator.functions import CSTRSimulator

# 1. Create simulator instance
sim = CSTRSimulator()

# 2. Define reaction
sim.reactions = [{
    "stoichiometry": {"A": -1, "B": 1},
    "reactants": ["A"],
    "products": ["B"],
    "frequency_factor": 1e10,           # A
    "activation_energy": 80000,       # Ea (J/mol)
    "reaction_order": {"A": 1},       # r = k * [A]
}]

# 3. Set required components
sim.components = ["A", "B"]
sim.feed_composition = {"A": 1.0, "B": 0.0}

# 4. Set reactor parameters (CORRECTED)
sim.set_parameters(
    feed_composition={"A": 1.0, "B": 0.0},
    reactions=sim.reactions,
    volume=1.0,         # m³
    temperature=350,    # K
    flow_rate=0.1       # m³/s
)

# 5. Solve for steady state
sim.solve_steady_state()

# 6. Display final concentrations
print("Steady-state concentrations:")
for comp, value in sim.concentrations.items():
    print(f"{comp} = {value:.4f} mol/L")

Steady-state concentrations:
B = 0.1149 mol/L
A = 0.8851 mol/L


### 5. `_verify_mass_balance`

🔍 Why it's important:

This internal method checks that the **elemental mass balance** is respected between the inlet and outlet streams.  
It is especially useful when multiple reactions or recycle streams are involved, and acts as a diagnostic tool to validate steady-state results.

In [52]:
# Use the internal function to verify mass balance at steady state

# Re-use inlet and outlet concentrations from earlier simulation
inlet = {"A": 1.0, "B": 0.0}
outlet = sim.concentrations          # steady-state concentrations
reaction = sim.reactions[0]          # current reaction in simulation

# Call internal verification method
sim._verify_mass_balance(inlet_conc=inlet, outlet_conc=outlet, reaction=reaction)

print("Mass balance verification completed successfully.")

Mass balance verification completed successfully.


### 6. `optimize_temperature`

🔍 Why it's important:

This function finds the **optimal temperature** that maximizes the yield of the target product.  
It uses a combination of **grid search** and **local optimization** to avoid getting stuck in poor local minima.

It calls:
- `solve_steady_state(temperature)` to simulate the reactor at each trial temperature,
- `calculate_yield()` to evaluate performance,
- and returns the temperature that gives the best result.

It is essential for analyzing **temperature sensitivity**, optimizing **reaction conditions**, and improving **reactor performance**.

In [54]:
from projet_chem200.cstr_simulator.functions import CSTRSimulator

# 1. Create simulator and define a reaction
sim = CSTRSimulator()
reaction = {
    "stoichiometry": {"A": -1, "B": 1},
    "reactants": ["A"],
    "products": ["B"],
    "frequency_factor": 1e10,
    "activation_energy": 80000,
    "reaction_order": {"A": 1},
    "reversible": False
}
sim.reactions = [reaction]

# 2. Set simulation parameters
sim.set_parameters(
    volume=1.0,
    temperature=300,
    flow_rate=0.1,
    reactions=sim.reactions,
    feed_composition={"A": 1.0, "B": 0.0},
    target_product="B"
)

# 3. Run temperature optimization
optimal_temp = sim.optimize_temperature(bounds=(300, 700))
print(f"Optimal temperature = {optimal_temp:.2f} K")

# 4. Show final yield at that temperature
final_yield = sim.calculate_yield()
print(f"Final yield at optimal T = {final_yield:.3f}")

Optimal temperature = 388.89 K
Final yield at optimal T = 1.000


### 7. `run_simulation`

🔍 Why it's important:

This is the **top-level simulation function**.  
It combines all key steps of the simulator:

- solving the steady state,
- optimizing the temperature,
- calculating product yield, reaction rates, and conversions,
- computing recycle and mass balance,
- and returning a complete summary of results.

It allows for **automated, end-to-end CSTR simulation** in one command.

In [56]:
from projet_chem200.cstr_simulator.functions import CSTRSimulator

# 1. Create a simulator instance
sim = CSTRSimulator()

# 2. Define the reaction
reaction = {
    "name": "A to B reaction",  # ✅ ajoute cette ligne
    "stoichiometry": {"A": -1, "B": 1},
    "reactants": ["A"],
    "products": ["B"],
    "frequency_factor": 1e10,
    "activation_energy": 80000,
    "reaction_order": {"A": 1},
    "reversible": False
}
sim.reactions = [reaction]

# 3. Set the parameters
sim.set_parameters(
    volume=1.0,
    temperature=300,
    flow_rate=0.1,
    reactions=sim.reactions,
    feed_composition={"A": 1.0, "B": 0.0},
    target_product="B"
)

# 4. Run the full simulation with optimization
results = sim.run_simulation(optimize_temp=True)

# 5. Print the results (optional)
sim.print_results(results)


===== CSTR SIMULATION RESULTS =====
Optimal Temperature: 455.56 K
Product Yield: 100.00%
Catalyst: None

Steady State Concentrations (mol/m³):
  B: 1.0000 (TARGET PRODUCT)
  A: 0.0000

Recycle Stream Concentrations (mol/m³):
  B: 0.0000
  A: 0.0000

Fresh Feed Concentrations (mol/m³):
  A: 1.0000
  B: 0.0000

Exit Flow Rate: 0.1000 m³/s
Residence Time: 10.00 s

Reaction Rates (mol/m³·s):
  Reaction 1: A to B reaction: 0.000000

Reactant Conversions:
  A: 100.00%

Mass Balance Error: 0.0000%

Mass Balance Check:
  Total Input Concentration: 1.0000 mol/m³
  Total Output Concentration: 1.0000 mol/m³
  Difference: 0.0000%



{'temperature': 455.55555555555554,
 'concentrations': {'B': 1.0, 'A': 0.0},
 'recycle_stream': {'B': 0.0, 'A': 0.0},
 'yield': 1,
 'reaction_rates': {'Reaction 1: A to B reaction': 6.710522079524489e-10},
 'conversions': {'A': 1.0},
 'residence_time': 10.0,
 'catalyst': None,
 'mass_balance_error': 0.0,
 'elemental_balance': {'total_input_conc': 1.0,
  'total_output_conc': 1.0,
  'difference_percent': 0.0}}